## Programmation concurrente ?

Au lieu d'exécuter les tâches les unes à la suite des autres, on les exécute les unes à côté des autres, voire les unes en même temps que les autres (programmation parallèle).

On distingue deux types de tâches :
* Les tâches *IO-bound*, qui dépendent principalement des entrées-sorties, pendant lesquelles le CPU passe le plus clair de son temps à attendre ;
* Les tâches *CPU-bound*, qui dépendent surtout du CPU.

La programmation concurrente permet de gagner en rapidité d'exécution :
* Pour les tâches *IO-bound*, un unique cœur peut exécuter les différentes tâches en profitant des temps d'attente ;
* Pour les tâches *CPU-bound*, on pourra répartir les différentes tâches sur différents cœurs.

## Quels outils (de la librairie standard) en Python ?
* `threading`
* `multiprocessing`
* `asyncio`

À noter : la bibliothèque `concurrent.futures` offre une interface plus simple à `threading` et `multiprocessing`

## La bibliothèque `threading`
Les différentes tâches sont réparties entre plusieurs *threads* qui se partagent la même mémoire virtuelle

In [1]:
import time
from typing import Callable, Any
from concurrent.futures import ThreadPoolExecutor


def doing_several_things_sequentially(n: int, action: Callable) -> []:
    res = list(map(action, range(n)))
    return res

def doing_several_things_with_threads(n: int, action: Callable) -> []:
    with ThreadPoolExecutor(max_workers=5) as executor:
        res = executor.map(action, range(n))
    return res

### Très efficace sur des tâches *IO-bound* ...

Définissons une simple tâche *IO-bound* : 

In [2]:
def doing_something_with_io(a: Any) -> Any:
    time.sleep(1) # Could be an I/O of some kind, like a call to a REST API...
    return a

#### Performances

Avec des threads :

In [3]:
%time doing_several_things_with_threads(5, doing_something_with_io);

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 1 s


Sans threads :

In [4]:
%time doing_several_things_sequentially(5, doing_something_with_io);

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 5 s


Les différents *threads* se répartissent le travail, le gain est important

### Beaucoup moins sur des tâches *CPU-bound* !

Définissons une tâche *CPU-bound*

In [5]:
def doing_something_with_the_cpu(a):
    [i**2 for i in range(4_000_000)]
    return a

#### Performance

Avec des threads

In [6]:
%time doing_several_things_with_threads(5, doing_something_with_the_cpu);

CPU times: user 5.05 s, sys: 922 ms, total: 5.97 s
Wall time: 6.01 s


Sans threads

In [7]:
%time doing_several_things_sequentially(5, doing_something_with_the_cpu);

CPU times: user 4.05 s, sys: 953 ms, total: 5 s
Wall time: 5 s


Surprise : la répartition du travail entre *threads* n'est plus du tout efficace !

### Tout ça à cause du GIL !
Le fait que les *threads* partagent la mémoire peut créér des problèmes. Pour les éviter CPython implémente une solution simple : le GIL.

Le *Global Interpreter Lock* (*GIL*) empêche deux *threads* de s'exécuter en même temps, même en présence de plusieurs cœurs.

Si les threads dépendent essentiellement du *CPU*, ils ne pourront pas s'exécuter en parallèle... et le gain sera inexistant !

Pire encore, le changement de contexte pourra même faire perdre un peu de temps !

## L'artillerie lourde pour ~résoudre~ contourner le problème : `multiprocessing`
On va créer plusieurs processus qui auront chacun leur propre instance de l'interpréteur, leur propre copie des données...

Plus de problème de GIL !

In [8]:
from concurrent.futures import ProcessPoolExecutor


def doing_several_things_with_multiprocess(n: int, action: Callable) -> []:
    with ProcessPoolExecutor(max_workers=5) as executor:
        res = executor.map(action, range(n))
    return res

### Pas de soucis avec les tâches *IO-bound*

#### Performances

Avec multiprocess

In [9]:
%time doing_several_things_with_multiprocess(5, doing_something_with_io);

CPU times: user 15.6 ms, sys: 31.2 ms, total: 46.9 ms
Wall time: 1.11 s


Sans multiprocess

In [10]:
%time doing_several_things_sequentially(5, doing_something_with_io);

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 5 s


Le gain est cependant moins important qu'avec `threading`

### Et cette fois ça fonctionne aussi pour des tâches *CPU-bound* !

#### Performances

Avec multiprocess

In [11]:
%time doing_several_things_with_multiprocess(5, doing_something_with_the_cpu);

CPU times: user 31.2 ms, sys: 93.8 ms, total: 125 ms
Wall time: 1.19 s


Sans multiprocess

In [12]:
%time doing_several_things_sequentially(5, doing_something_with_the_cpu);

CPU times: user 4.17 s, sys: 844 ms, total: 5.02 s
Wall time: 5 s


Cette fois-ci, vu que chaque process dispose de son propre interpréteur, le GIL ne nous bloque plus. Les tâches s'exécutent en parrallèle sur plusieurs cœurs.

### Inconvénients

L'utilisation de plusieurs processus a un impact sur la consommation de mémoire. En plus de cela, les opérations de création / synchronisation des différents processus sont coûteuses. Résultat, `multiprocess` est moins efficace que `threading` sur les tâches *IO-bound*

Avec multiprocess

In [13]:
%timeit doing_several_things_with_multiprocess(5, doing_something_with_io);

1.11 s ± 6.03 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


Avec des threads

In [14]:
%timeit doing_several_things_with_threads(5, doing_something_with_io);

1 s ± 487 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


## Une autre approche à la mode : `asyncio`

Plus de *threads* ou de processus, c'est l'interpréteur qui gère tout !

In [15]:
import asyncio


async def doing_several_things_with_asyncio(n: int, action: Callable) -> []:
    await asyncio.gather(*[action(i) for i in range(n)])

### Efficace pour des tâches *IO-bound*

In [16]:
async def doing_something_with_io_awaitable(a: Any) -> Any:
    await asyncio.sleep(1)
    return a

async def doing_something_with_cpu_awaitable(a: Any) -> Any:
    [i**2 for i in range(4_000_000)]
    return a

#### Performances

In [17]:
start_time = time.time()
await doing_several_things_with_asyncio(5, doing_something_with_io_awaitable) # En dehors de Jupyter, il faudrait écrire asyncio.run(doing_several_things_with_asyncio(5, doing_something_with_io_awaitable))
end_time = time.time()

print(f'elapsed time with asyncio: {end_time - start_time}')

elapsed time with asyncio: 1.0010147094726562


In [18]:
start_time = time.time()
doing_several_things_with_threads(5, doing_something_with_io)
end_time = time.time()

print(f'elapsed time sequentially: {end_time - start_time}')

elapsed time sequentially: 1.0033788681030273


### Mais pas pour des tâches *CPU-bound* (mais on s'en doutait)

In [19]:
start_time = time.time()
await doing_several_things_with_asyncio(5, doing_something_with_cpu_awaitable)
end_time = time.time()

print(f'elapsed time with asyncio: {end_time - start_time}')

elapsed time with asyncio: 5.1629862785339355


L'interpréteur n'utilise qu'un seul *thread* pour tout exécuter, impossible de faire mieux que du séquentiel !

## Et dans le futur

Et si on supprimait le GIL pour régler tous les problèmes ?

C'est prévu : la [PEP 703](https://peps.python.org/pep-0703/) prévoit de rendre le GIL optionnel ! Le *Steering Council* a annoncé [en juillet](https://discuss.python.org/t/a-steering-council-notice-about-pep-703-making-the-global-interpreter-lock-optional-in-cpython/30474/1) son intention d'accepter cette PEP, à priori selon le calendrier suivant :
* Un build no-GIL expérimental est prévu pour la version 3.13 ou 3.14 (2024 ou 2025)
* Après environ un ou deux ans, le mode no-GIL sera supporté officiellement
* Avant de devenir le mode par défaut

Mais le travail est conséquent puisque c'est toute la gestion de la mémoire de CPython qu'il faut revoir ! 